<a href="https://colab.research.google.com/github/ieg-dhr/DigiKAR/blob/main/JupyterNotebooks_DigiKAR/PROFs_duplicateROWifDATES.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is a script for transferring PROF & STUDENT data from TXT to EXCEL while identifying known entities from the DigiKAR ontology lists.

First of all, we need to connect this Colab notebook with your Google Drive and define the directory for input and output data.


In [1]:
## mount drive
from google.colab import drive
drive.mount("/content/drive")
directory="/content/drive/My Drive/Colab_DigiKAR/"

Mounted at /content/drive


In [2]:
!pip install xlsxwriter
!pip install pandas
!pip install numpy

import xlsxwriter
import numpy
import pandas as pd
import re

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.0/153.0 kB 4.9 MB/s eta 0:00:00


In [3]:
# Read full dataframe
data = "https://github.com/ieg-dhr/DigiKAR/raw/main/Sample%20Data/Factoid-PROFS-consolidation-STEP3-events-split-v10.xlsx"
df_all = pd.read_excel(data, index_col=None, dtype=str)

# Create sub-dataframe where values contain hashtags

# Creating sub-DataFrame where a cell contains a '#'
sub_df = df_all[df_all['event_after-date'].str.contains('#') | df_all['event_before-date'].str.contains('#') | df_all['event_start'].str.contains('#') | df_all['event_end'].str.contains('#') | df_all['event_date'].str.contains('#')]

display(sub_df)

/usr/local/lib/python3.10/dist-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,factoid_ID,pers_ID,pers_name,alternative_names,event_type,event_after-date,event_before-date,event_start,event_end,event_date,...,pers_function,place_name,inst_name,rel_pers,source_quotations,additional_info,comment,info_dump,source,source_site
271,6602,OCR,Adam Nydaenus,Nydenus,Funktionsausübung,NaN,NaN,1578 # 1579 # 1580,1578 # 1579 # 1580,NaN,...,Lehrer,Mainz,"Universität Mainz, Philosophische Fakultät",NaN,NaN,er lehrte in Mainz an der Artistenfakultät 157...,NaN,NaN,"Cat. Jes.1597, Nr.5; \n Nom. rev., S.3, 26; \...",NaN
407,358,OCR,Albert vom Putz,de Puteo,Funktionsausübung,NaN,NaN,1603 # 1604,1603 # 1604,NaN,...,Assessor,"Mainz, Königstein","Universität Mainz, Theologische Fakultät",NaN,NaN,von Mitte August bis Ende Dezember 1603 war er...,NaN,NaN,"Praetorius, Professoren, S.94 hier: + 25. 9.1...",NaN
408,358,OCR,Albert vom Putz,de Puteo,Funktionsausübung,NaN,NaN,1603 # 1604,1603 # 1604,NaN,...,Reisender,"Mainz, Königstein","Universität Mainz, Theologische Fakultät",NaN,NaN,von Mitte August bis Ende Dezember 1603 war er...,NaN,NaN,"Praetorius, Professoren, S.94 hier: + 25. 9.1...",NaN
411,353,OCR,Albert vom Putz,de Puteo,Promotion,NaN,NaN,1595 # 1596,1595 # 1596,NaN,...,Promovend,NaN,NaN,NaN,NaN,1595 zum Baccalaureus bibl. und 1596 zum Dr. t...,NaN,NaN,"Praetorius, Professoren, S.94 hier: + 25. 9.1...",NaN
439,reconstruction,OCR,Alexander Dietrich,Theodorici,Funktionsausübung,NaN,NaN,1495 # 1496,NaN,1495 # 1496,...,Prokurator,NaN,Reichskammergericht,NaN,NaN,"1495 Advokat, 1496 Prokurator am Reichskammerg...",NaN,NaN,"Praetorius, Professoren, S.96; \n Amrhein, S....",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12803,3313,OCR,Wolfgang Ernst Heidel,NaN,Funktionsausübung,NaN,NaN,1695 # 1698 # 1699,1695 # 1698 # 1699,NaN,...,Rektor,Mainz,Universität Mainz],NaN,NaN,er war auch Universitätskanzler (z.B. 1685) un...,NaN,NaN,"Praetorius, Professoren, S.99; \n Prot. jur. ...",NaN
12807,6898,OCR,Wolfgang Schwan,NaN,Funktionsausübung,NaN,NaN,1676 # 1679 # 1683,1676 # 1679 # 1683,NaN,...,Rektor,"Speyer, Mainz",Jesuiten Oberrhein,NaN,NaN,"1676 war er Rektor des Kollegs in Speyer, 1679...",NaN,NaN,"Praetorius, Professoren, S.136; \n Cat. Jes. ...",NaN
12808,6898,OCR,Wolfgang Schwan,NaN,Funktionsausübung,NaN,NaN,1676 # 1679 # 1683,1676 # 1679 # 1683,NaN,...,Rektor,"Speyer, Mainz",Jesuiten Oberrhein,NaN,NaN,"1676 war er Rektor des Kollegs in Speyer, 1679...",NaN,NaN,"Praetorius, Professoren, S.136; \n Cat. Jes. ...",NaN
12809,6898,OCR,Wolfgang Schwan,NaN,Funktionsausübung,NaN,NaN,1676 # 1679 # 1683,1676 # 1679 # 1683,NaN,...,Rektor,"Speyer, Mainz",Jesuiten Oberrhein,NaN,NaN,"1676 war er Rektor des Kollegs in Speyer, 1679...",NaN,NaN,"Praetorius, Professoren, S.136; \n Cat. Jes. ...",NaN


In [7]:
keywords=["und", "oder", "später", "außerdem", "danach"]
myRange = range(1500, 1900)
myColumns = ["factoid_ID",
                "pers_ID",
                "pers_name",
                "alternative_names",
                "event_type",
                "event_after-date",
                "event_before-date",
                "event_start",
                "event_end",
                "event_date",
                "pers_title",
                "pers_function",
                "place_name",
                "inst_name",
                "rel_pers",
                "source_quotations",
                "additional_info",
                "comment",
                "info_dump",
                "source",
                "source_site"]

def duplicate_rows_by_year(df):
    rows_to_duplicate = []

    for index, row in df.iterrows():
        years = years = re.split(r"[ ,;.]()", row['additional_info'])
        print(years)
        for year in years:
            if year in keywords:
                rows_to_duplicate.append(row)
                print("keyword found")
            elif year.isdigit() == True:
                if (int(year)) in myRange:
                    rows_to_duplicate.append(row)
                    print("integer found")

                else:
                  print("no integers found")
            else:
                print("no copying needed")

    print(rows_to_duplicate)
    df_extended = pd.DataFrame(rows_to_duplicate, columns=myColumns)
    #df_extended = pd.concat([sub_df] + rows_to_duplicate, ignore_index=True)
    return df_extended

# Display final df
df_extended = duplicate_rows_by_year(sub_df)
display(df_extended)


Streaming output truncated to the last 5000 lines.
no copying needed
no copying needed
no copying needed
no copying needed
no copying needed
no copying needed
no copying needed
no copying needed
no copying needed
no copying needed
no copying needed
integer found
no copying needed
no copying needed
no copying needed
no copying needed
no copying needed
no copying needed
no copying needed
no copying needed
no copying needed
no copying needed
no copying needed
no copying needed
no copying needed
keyword found
no copying needed
no copying needed
no copying needed
keyword found
no copying needed
integer found
no copying needed
no copying needed
no copying needed
no copying needed
no copying needed
no copying needed
['Gau-Odernheim', '', 'Magister', '', '', '', '1560', '', 'sowie', '', '1579', '', 'Dekan', '', 'der', '', 'Artistenfakultät', '', 'in', '', 'Mainz']
no copying needed
no copying needed
no copying needed
no copying needed
no copying needed
no copying needed
integer found
no copyin

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



,factoid_ID,pers_ID,pers_name,alternative_names,event_type,event_after-date,event_before-date,event_start,event_end,event_date,...,pers_function,place_name,inst_name,rel_pers,source_quotations,additional_info,comment,info_dump,source,source_site
271,6602,OCR,Adam Nydaenus,Nydenus,Funktionsausübung,NaN,NaN,1578 # 1579 # 1580,1578 # 1579 # 1580,NaN,...,Lehrer,Mainz,"Universität Mainz, Philosophische Fakultät",NaN,NaN,er lehrte in Mainz an der Artistenfakultät 157...,NaN,NaN,"Cat. Jes.1597, Nr.5; \n Nom. rev., S.3, 26; \...",NaN
271,6602,OCR,Adam Nydaenus,Nydenus,Funktionsausübung,NaN,NaN,1578 # 1579 # 1580,1578 # 1579 # 1580,NaN,...,Lehrer,Mainz,"Universität Mainz, Philosophische Fakultät",NaN,NaN,er lehrte in Mainz an der Artistenfakultät 157...,NaN,NaN,"Cat. Jes.1597, Nr.5; \n Nom. rev., S.3, 26; \...",NaN
271,6602,OCR,Adam Nydaenus,Nydenus,Funktionsausübung,NaN,NaN,1578 # 1579 # 1580,1578 # 1579 # 1580,NaN,...,Lehrer,Mainz,"Universität Mainz, Philosophische Fakultät",NaN,NaN,er lehrte in Mainz an der Artistenfakultät 157...,NaN,NaN,"Cat. Jes.1597, Nr.5; \n Nom. rev., S.3, 26; \...",NaN
407,358,OCR,Albert vom Putz,de Puteo,Funktionsausübung,NaN,NaN,1603 # 1604,1603 # 1604,NaN,...,Assessor,"Mainz, Königstein","Universität Mainz, Theologische Fakultät",NaN,NaN,von Mitte August bis Ende Dezember 1603 war er...,NaN,NaN,"Praetorius, Professoren, S.94 hier: + 25. 9.1...",NaN
407,358,OCR,Albert vom Putz,de Puteo,Funktionsausübung,NaN,NaN,1603 # 1604,1603 # 1604,NaN,...,Assessor,"Mainz, Königstein","Universität Mainz, Theologische Fakultät",NaN,NaN,von Mitte August bis Ende Dezember 1603 war er...,NaN,NaN,"Praetorius, Professoren, S.94 hier: + 25. 9.1...",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12809,6898,OCR,Wolfgang Schwan,NaN,Funktionsausübung,NaN,NaN,1676 # 1679 # 1683,1676 # 1679 # 1683,NaN,...,Rektor,"Speyer, Mainz",Jesuiten Oberrhein,NaN,NaN,"1676 war er Rektor des Kollegs in Speyer, 1679...",NaN,NaN,"Praetorius, Professoren, S.136; \n Cat. Jes. ...",NaN
12809,6898,OCR,Wolfgang Schwan,NaN,Funktionsausübung,NaN,NaN,1676 # 1679 # 1683,1676 # 1679 # 1683,NaN,...,Rektor,"Speyer, Mainz",Jesuiten Oberrhein,NaN,NaN,"1676 war er Rektor des Kollegs in Speyer, 1679...",NaN,NaN,"Praetorius, Professoren, S.136; \n Cat. Jes. ...",NaN
12809,6898,OCR,Wolfgang Schwan,NaN,Funktionsausübung,NaN,NaN,1676 # 1679 # 1683,1676 # 1679 # 1683,NaN,...,Rektor,"Speyer, Mainz",Jesuiten Oberrhein,NaN,NaN,"1676 war er Rektor des Kollegs in Speyer, 1679...",NaN,NaN,"Praetorius, Professoren, S.136; \n Cat. Jes. ...",NaN
12816,2104,OCR,Wolfgang Speth,NaN,Funktionsausübung,NaN,NaN,1644 # 1647,1644 # 1647,NaN,...,Rektor,Bamberg,Jesuiten ###,NaN,NaN,vom November 1644 bis Ende Oktober 1647 war er...,NaN,NaN,"De Back.-Som. VII, Sp.1436; \n Wegele, S.352;...",NaN


In [9]:

# write all results to new EXCEL file

workbook=directory+'FACTOIDS_consolidated/Factoid_PROFS_hashtags_v2.xlsx'
print(workbook)
writer = pd.ExcelWriter(workbook, engine='xlsxwriter') # create a Pandas Excel writer using XlsxWriter as the engine.
df_extended.to_excel(writer, sheet_name='FactCons') # Convert the dataframe to an XlsxWriter Excel object.
writer.save() # Close the Pandas Excel writer and output the Excel file.
print("Done.")

/content/drive/My Drive/Colab_DigiKAR/FACTOIDS_consolidated/Factoid_PROFS_hashtags_v2.xlsx


<ipython-input-9-93aa67edc2cc>:7: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save() # Close the Pandas Excel writer and output the Excel file.


Done.


Now we can import the packages to the script and load our data.

Check the output files and repeat process with refined ontology files if necessary.

Script by Monika Barget, Maastricht/Mainz

January 2023
